# Kubernetes Maintenance Operations

## Rolling upgrade of our deployment

What happens when we need to provide a new version of our application?

We need the platform to handle such upgrades cluster wide automatically.

For this we use the rolling upgrade capability

#### Let's first review our current deployment '*k8s-demo*'

In [6]:
kubectl get deploy k8s-demo

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
k8s-demo   2         2         2            2           10h


In [7]:
kubectl describe deploy k8s-demo

Name:                   k8s-demo
Namespace:              default
CreationTimestamp:      Thu, 18 Jan 2018 21:00:50 +0000
Labels:                 app=k8s-demo
Annotations:            deployment.kubernetes.io/revision=1
Selector:               app=k8s-demo
Replicas:               2 desired | 2 updated | 2 total | 2 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  1 max unavailable, 1 max surge
Pod Template:
  Labels:  app=k8s-demo
  Containers:
   k8s-demo:
    Image:        mjbright/k8s-demo:1
    Port:         8080/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
OldReplicaSets:  <none>
NewReplicaSet:   k8s-demo-7bc74f7946 (2/2 replicas created)
Events:          <none>


We see above that we are running version 1 of our service.

In [8]:
kubectl describe deploy k8s-demo | grep Image:

    Image:        mjbright/k8s-demo:1


Once a new version is available to deploy we perform a rolling upgrade to roll out this new version:

We set the image attribute of our deployment.

We can perform a '*rollout status*' to see the upgrade as it processes

In [9]:
kubectl set image deployment k8s-demo k8s-demo=mjbright/k8s-demo:2 --record


kubectl rollout status deploy k8s-demo

deployment "k8s-demo" image updated
Waiting for rollout to finish: 1 old replicas are pending termination...
Waiting for rollout to finish: 1 old replicas are pending termination...
Waiting for rollout to finish: 1 old replicas are pending termination...
Waiting for rollout to finish: 1 old replicas are pending termination...
deployment "k8s-demo" successfully rolled out


#### Get new pods info

In [11]:
kubectl get pods -o wide

POD1=$(kubectl get pods -o wide --no-headers=true | head -1 | awk '{print $1;}')
POD2=$(kubectl get pods -o wide --no-headers=true | tail -1 | awk '{print $1;}')
POD1_IP=$(kubectl get pods -o wide --no-headers=true | head -1 | awk '{print $6;}')
POD2_IP=$(kubectl get pods -o wide --no-headers=true | tail -1 | awk '{print $6;}')

echo POD1_IP=$POD1_IP
echo POD2_IP=$POD2_IP

NAME                       READY     STATUS    RESTARTS   AGE       IP           NODE
k8s-demo-88b6b8dbf-245pk   1/1       Running   0          4m        10.192.3.2   kube-node-2
k8s-demo-88b6b8dbf-fxmc6   1/1       Running   0          4m        10.192.2.2   kube-node-1
POD1_IP=10.192.3.2
POD2_IP=10.192.2.2


### Observing the new version

We can now access our application to verify that the deployment has been upgraded.

In [14]:
curl $POD1_IP:8080


                ./shhs:`                
            .+ymmdyosydNms/`            
        -+yNmhs+++++++++oydNmy/.        
    .+hNmhs++++++++yy+++++++oydNmy+.    
   +Mds++++++++++++md++++++++++++smM/   
  `MN++++++++++oshhNNhys+++++++++++Nm   
  oMy++++mmsohNMNddMMhdNMmy+ymm++++yM/  
  NN++++++omMMmo++sMM+++sNMMdo++++++Nd  
 /Mh++++++yMNdMNhsyMMssdMMdNMs++++++hM- 
 mMo++++++NMo+ohMMMNNMMNh++sMm++++++oMy 
:Md++++++oMNosydNMm++NMNhysoMM+++++++hM.
hMo+++shhdMMNNmddMMNNMNddmNMMMdhyo+++oMs
mMo+++syo+sMmo++sMMhdMNo++oNMy+oss+++oMh
-mMy+++++++sNNysMMy++hMNohMNs+++++++yNh`
  oMmo+++++++ymMMmhyyhNMMmy+++++++omm:  
   .hMh+++++++hNyyhddhyymh+++++++hMo`   
     /NNs++++yNo++++++++oNy++++sNd.     
      `sMdo++++++++++++++++++odN/       
        -dMhsssssssssssssssshNy`        
          :shhhhhhhhhhhhhhhyo.          


Served from container k8s-demo-88b6b8dbf-245pk


In [15]:
curl $POD2_IP:8080


                ./shhs:`                
            .+ymmdyosydNms/`            
        -+yNmhs+++++++++oydNmy/.        
    .+hNmhs++++++++yy+++++++oydNmy+.    
   +Mds++++++++++++md++++++++++++smM/   
  `MN++++++++++oshhNNhys+++++++++++Nm   
  oMy++++mmsohNMNddMMhdNMmy+ymm++++yM/  
  NN++++++omMMmo++sMM+++sNMMdo++++++Nd  
 /Mh++++++yMNdMNhsyMMssdMMdNMs++++++hM- 
 mMo++++++NMo+ohMMMNNMMNh++sMm++++++oMy 
:Md++++++oMNosydNMm++NMNhysoMM+++++++hM.
hMo+++shhdMMNNmddMMNNMNddmNMMMdhyo+++oMs
mMo+++syo+sMmo++sMMhdMNo++oNMy+oss+++oMh
-mMy+++++++sNNysMMy++hMNohMNs+++++++yNh`
  oMmo+++++++ymMMmhyyhNMMmy+++++++omm:  
   .hMh+++++++hNyyhddhyymh+++++++hMo`   
     /NNs++++yNo++++++++oNy++++sNd.     
      `sMdo++++++++++++++++++odN/       
        -dMhsssssssssssssssshNy`        
          :shhhhhhhhhhhhhhhyo.          


Served from container k8s-demo-88b6b8dbf-fxmc6


#### Blue-green upgrades

Note that if we performed these tests during the upgrade we might see the old "blue" icon and the new "red" icon whilst some pods have been upgraded and others not.

## TO ADD

- Readystatus and health checks
- Rollback
- Node cordoning